<a href="https://colab.research.google.com/github/jdmartinezrs/MachineLearningAlgoritms/blob/main/Predicci%C3%B3nDeDatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor  # Cambiado a Regressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
sheet_id = "1r4szNXN8abPstdpZEtuePc89H6AJPgjq"
sheet_url = f"https://drive.google.com/uc?id={sheet_id}&export=csv"
df = pd.read_csv(sheet_url)

In [12]:
df.rename(columns={
    'Ult Km/Hr': 'Ult_Km',
    'Tipo de Vehiculo': 'Tipo_Vehiculo',
    'Km/Hr Pend': 'Km_Pend',
    'Progreso': 'Progreso'
}, inplace=True)

In [13]:
features = ['Ult_Km', 'Tipo_Vehiculo', 'Progreso']
target = 'Km_Pend'

X = df[features]
y = df[target]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
numeric_features = ['Ult_Km', 'Progreso']
categorical_features = ['Tipo_Vehiculo']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)  # Manejo de categorías desconocidas
    ])

# Crear pipeline con preprocesamiento y modelo
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', KNeighborsRegressor(n_neighbors=3))  # Usando Regressor en lugar de Classifier
])

# Entrenar el modelo
model.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  ['Ult_Km', 'Progreso']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Tipo_Vehiculo'])])),
                ('regressor', KNeighborsRegressor(n_neighbors=3))])

In [16]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print(f'MAE: {mae:.2f}')
    print(f'MSE: {mse:.2f}')
    print(f'RMSE: {rmse:.2f}')
    print(f'R2 Score: {r2:.2f}')

    return y_pred

In [17]:
def evaluate_model(model, X_test, y_test):
    # Hacer predicciones
    y_pred = model.predict(X_test)

    # Calcular métricas
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print(f'MAE: {mae:.2f}')
    print(f'MSE: {mse:.2f}')
    print(f'RMSE: {rmse:.2f}')
    print(f'R2 Score: {r2:.2f}')

    return y_pred

# Evaluar el modelo
print("Evaluación del modelo:")
y_pred = evaluate_model(model, X_test, y_test)


Evaluación del modelo:
MAE: 216.70
MSE: 83301.03
RMSE: 288.62
R2 Score: 0.99


In [18]:
def predict_new_data(model):
    # Datos de ejemplo con las categorías correctas
    new_data = pd.DataFrame({
        'Ult_Km': [5000, 10000, 15000],
        'Tipo_Vehiculo': ['VOLQ SENCILLA', 'DOBLETROQUE', 'CARRO TANQUE'],
        'Progreso': [0.5, 0.7, 0.9]
    })

    try:
        predictions = model.predict(new_data)
        new_data['Predicted_Km_Pend'] = predictions
        return new_data
    except Exception as e:
        print(f"Error al predecir: {str(e)}")
        # Mostrar categorías únicas en los datos de entrenamiento
        print("\nCategorías válidas para 'Tipo_Vehiculo':")
        print(X_train['Tipo_Vehiculo'].unique())
        return None

# Probar con nuevos datos
print("\nPredicciones para nuevos datos:")
new_predictions = predict_new_data(model)
if new_predictions is not None:
    print(new_predictions)


Predicciones para nuevos datos:
   Ult_Km  Tipo_Vehiculo  Progreso  Predicted_Km_Pend
0    5000  VOLQ SENCILLA       0.5        5725.666667
1   10000    DOBLETROQUE       0.7        5690.000000
2   15000   CARRO TANQUE       0.9        8961.333333
